In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import argparse
import datasets
import pandas
import transformers
import tensorflow as tf
import numpy

# use the tokenizer from BERT
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(examples):
    """Tokenize the input text and return input_ids and attention_mask."""
    tokens = tokenizer(examples["text"], truncation=True, max_length=64, padding="max_length")
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"]
    }

def train(model_path="model",
          train_path="/content/drive/MyDrive/Colab Notebooks/train.csv",
          dev_path="/content/drive/MyDrive/Colab Notebooks/dev.csv"):

    # Load CSVs into HuggingFace datasets
    hf_dataset = datasets.load_dataset("csv", data_files={
        "train": train_path,
        "validation": dev_path
    })

    # Define label names
    labels = hf_dataset["train"].column_names[1:]  # First column is 'text'

    def gather_labels(example):
        """Combine all label columns into a single list."""
        return {"labels": [float(example[l]) for l in labels]}

    # Preprocess: label formatting + tokenization
    hf_dataset = hf_dataset.map(gather_labels)
    hf_dataset = hf_dataset.map(tokenize, batched=True)

    # Convert HuggingFace dataset to TensorFlow dataset
    train_dataset = hf_dataset["train"].to_tf_dataset(
        columns=["input_ids", "attention_mask"],
        label_cols="labels",
        batch_size=16,
        shuffle=True
    )
    dev_dataset = hf_dataset["validation"].to_tf_dataset(
        columns=["input_ids", "attention_mask"],
        label_cols="labels",
        batch_size=16
    )

    # Load pretrained BERT base model (without classification head)
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")

    # Input layers
    input_ids = tf.keras.Input(shape=(64,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(64,), dtype=tf.int32, name="attention_mask")

    # BERT output returns a tuple with multiple outputs (depending on the BERT variant you're using)
    # output = (last_hidden_state, pooled_output, hidden_states, attentions)
    # [0] gives last hidden state, This contains the contextualized embeddings for every token in the input
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

    # Pooling
    pooled_output = tf.keras.layers.GlobalAveragePooling1D()(bert_output)
    x = tf.keras.layers.LayerNormalization()(pooled_output)

    # Dense head for multi-label classification
    dropout = tf.keras.layers.Dropout(0.4)(x)
    dense = tf.keras.layers.Dense(256, activation="relu",
                                   kernel_regularizer=tf.keras.regularizers.l2(0.0005))(dropout)
    output = tf.keras.layers.Dense(len(labels), activation="sigmoid")(dense)

    # Final model
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    # Compile
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
        metrics=[tf.keras.metrics.F1Score(average="micro", threshold=0.5)]
    )

    # Callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
    )
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, verbose=1
    )
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=f"{model_path}.keras",
        monitor="val_f1_score", mode="max", save_best_only=True, verbose=1
    )
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs", histogram_freq=1)

    # Train
    model.fit(
        train_dataset,
        validation_data=dev_dataset,
        epochs=4,
        callbacks=[early_stopping, lr_scheduler, model_checkpoint, tensorboard]
    )

train()

Map:   0%|          | 0/3149 [00:00<?, ? examples/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/4


1575/1575 [==============================] - ETA: 0s - loss: 0.3673 - f1_score: 0.7719
Epoch 1: val_f1_score improved from -inf to 0.83754, saving model to model.keras


/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


1575/1575 [==============================] - 477s 278ms/step - loss: 0.3673 - f1_score: 0.7719 - val_loss: 0.3379 - val_f1_score: 0.8375 - lr: 2.0000e-05
Epoch 2/4
1575/1575 [==============================] - ETA: 0s - loss: 0.3209 - f1_score: 0.8490
Epoch 2: val_f1_score improved from 0.83754 to 0.85070, saving model to model.keras
1575/1575 [==============================] - 427s 271ms/step - loss: 0.3209 - f1_score: 0.8490 - val_loss: 0.3100 - val_f1_score: 0.8507 - lr: 2.0000e-05
Epoch 3/4
1575/1575 [==============================] - ETA: 0s - loss: 0.2846 - f1_score: 0.8905
Epoch 3: val_f1_score did not improve from 0.85070
1575/1575 [==============================] - 411s 261ms/step - loss: 0.2846 - f1_score: 0.8905 - val_loss: 0.2855 - val_f1_score: 0.8412 - lr: 2.0000e-05
Epoch 4/4
1575/1575 [==============================] - ETA: 0s - loss: 0.2503 - f1_score: 0.9281
Epoch 4: val_f1_score did not improve from 0.85070
1575/1575 [==============================] - 420s 266ms/step 